<a href="https://colab.research.google.com/github/karthiqak47/DSP_Learning/blob/main/DSP_LAB_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1.Import sample audio

# 2. Plot Autocorrelation

# 3.Second Peak of autocorrelation

# 4.Estimated Attenuation

In [1]:
%%writefile c_program15.c
#include <stdio.h>
#include <math.h>

#define N 8
#define PI 3.141592653589793

typedef struct {
    double re;
    double im;
} Complex;


void printArray(Complex *x, int n, const char *msg) {
    printf("\n%s\n", msg);
    for (int i = 0; i < n; i++) {
        if (fabs(x[i].im) < 1e-29)
            x[i].im = 0;
        printf("X[%d] = %8.4f + j%8.4f\n", i, x[i].re, x[i].im);
    }
}


unsigned int reverseBits(unsigned int num, int log2n) {
    unsigned int result = 0;
    for (int i = 0; i < log2n; i++) {
        if (num & (1 << i))
            result |= 1 << ((log2n - 1) - i);
    }
    return result;
}


void bitReverse(Complex *x, int n) {
    int log2n = (int)(log(n) / log(2));
    for (unsigned int i = 0; i < n; i++) {
        unsigned int j = reverseBits(i, log2n);
        if (i < j) {
            Complex temp = x[i];
            x[i] = x[j];
            x[j] = temp;
        }
    }
}


void fftdit(Complex *x, int n, int op) {
    int log2n, half,stride,b,j, m;
    int s;
    Complex wn[N / 2];
    printArray(x, n, "I/p");
    bitReverse(x,n);
    printArray(x, n, "after bit reverse");
    log2n=(int)(log(n)/log(2));



    for (s = 0; s < log2n; s++) {
        m = pow(2, s + 1);
        half = m / 2.0;
        stride = N / m;

        printf("Stage %i -> m = %i,\thalf = %i,\tstride = %i\n", s, m, half, stride);

        for (j = 0; j < half; j++) {

            if (op == 1) {
                wn[j].im = sin(2 * PI/N * j * stride);
                wn[j].re = cos(2 * PI/N * j * stride);
            } else {
                wn[j].im = (double)-sin(2 * PI/N * j * stride);
                wn[j].re = (double)cos(2 * PI/N * j * stride);
            }

        }

        for (b = 0; b < N; b += m) {
            for (j = 0; j < half; j++) {
                // (j + b)
                // (j + b + half)
                // printf("(%f,\t%f)\n", x[j + b].re, x[j + b + half].re);

                Complex t = {
                    wn[j].re * x[j + b + half].re - wn[j].im * x[j + b + half].im,
                    wn[j].re * x[j + b + half].im + wn[j].im * x[j + b + half].re
                };


                x[j + b + half].re = x[j + b].re - t.re;
                x[j + b + half].im = x[j + b].im - t.im;

                x[j + b].re += t.re;
                x[j + b].im += t.im;

                printf("(%i,\t%i) + (%f,%f) -> (%f + j%f | %f + j%f)\n", j + b, j + b + half, wn[j].re, wn[j].im, x[j + b].re, x[j + b].im, x[j + b + half].re, x[j + b + half].im);
            }
        }

        printArray(x, n, "stage output");
    }
}

void main() {
    Complex x[N] = {
        { 1, 1 },
        { 2, -1 },
        { -1, 2 },
        { 0, -3 },
        { 3, 0 },
        { -2, 1 },
        { 1, -2 },
        { -1, 0 }
    };


  // fftdit(x, N, 0); // fft
  fftdit(x, N, 0); // ifft
  fftdit(x, N, 1); // ifft
}

Writing c_program15.c


In [2]:
%%shell
gcc c_program15.c -o c_program -lm
./c_program


I/p
X[0] =   1.0000 + j  1.0000
X[1] =   2.0000 + j -1.0000
X[2] =  -1.0000 + j  2.0000
X[3] =   0.0000 + j -3.0000
X[4] =   3.0000 + j  0.0000
X[5] =  -2.0000 + j  1.0000
X[6] =   1.0000 + j -2.0000
X[7] =  -1.0000 + j  0.0000

after bit reverse
X[0] =   1.0000 + j  1.0000
X[1] =   3.0000 + j  0.0000
X[2] =  -1.0000 + j  2.0000
X[3] =   1.0000 + j -2.0000
X[4] =   2.0000 + j -1.0000
X[5] =  -2.0000 + j  1.0000
X[6] =   0.0000 + j -3.0000
X[7] =  -1.0000 + j  0.0000
Stage 0 -> m = 2,	half = 1,	stride = 4
(0,	1) + (1.000000,-0.000000) -> (4.000000 + j1.000000 | -2.000000 + j1.000000)
(2,	3) + (1.000000,-0.000000) -> (0.000000 + j0.000000 | -2.000000 + j4.000000)
(4,	5) + (1.000000,-0.000000) -> (0.000000 + j0.000000 | 4.000000 + j-2.000000)
(6,	7) + (1.000000,-0.000000) -> (-1.000000 + j-3.000000 | 1.000000 + j-3.000000)

stage output
X[0] =   4.0000 + j  1.0000
X[1] =  -2.0000 + j  1.0000
X[2] =   0.0000 + j  0.0000
X[3] =  -2.0000 + j  4.0000
X[4] =   0.0000 + j  0.0000
X[5] =   4.00